In [3]:
#!/usr/bin/env python
from PyDAQmx.DAQmxFunctions import *
from PyDAQmx.DAQmxConstants import *
import numpy as np
import matplotlib.pyplot as plt

In [13]:
class MultiChannelAnalogInput(object):
    """Class to create a multi-channel analog input
    
    Usage: AI = MultiChannelInput(physicalChannel)
        physicalChannel: a string or a list of strings
    optional parameter: limit: tuple or list of tuples, the AI limit values
                        reset: Boolean
    Methods:
        read(name), return the value of the input name
        readAll(), return a dictionary name:value
    """
    def __init__(self,physicalChannel, limit = None, reset = False):
        self.taskHandle = None
        if type(physicalChannel) == type(""):
            self.physicalChannel = [physicalChannel]
        else:
            self.physicalChannel  =physicalChannel
        self.nCh = physicalChannel.__len__()
        if limit is None:
            self.limit = dict([(name, (-10.0,10.0)) for name in self.physicalChannel])
        elif type(limit) == tuple:
            self.limit = dict([(name, limit) for name in self.physicalChannel])
        else:
            self.limit = dict([(name, limit[i]) for  i,name in enumerate(self.physicalChannel)])           
        if reset:
            DAQmxResetDevice(physicalChannel[0].split('/')[0] )
            
    def configure(self, address , rate=10000., nSample=1E3, trig=None, trigSlopePositive=True, trigLevel=0.0):
        # Create one task handle for all
        self.rate = rate
        self.nSample = nSample
        self.taskHandle = TaskHandle(0)
        DAQmxCreateTask("",byref(self.taskHandle))
        for name in self.physicalChannel:
            DAQmxCreateAIVoltageChan(self.taskHandle,name,"",DAQmx_Val_RSE,
                                     self.limit[name][0],self.limit[name][1],
                                     DAQmx_Val_Volts,None)
        print(nSample)
        DAQmxCfgSampClkTiming(self.taskHandle,"",float(rate),DAQmx_Val_Rising,DAQmx_Val_FiniteSamps,int(nSample))
        if trig is None:
            DAQmxDisableStartTrig(self.taskHandle)
        elif str(trig).startswith('%s/port' % address):
            # digital trigger
            trigEdge = DAQmx_Val_Rising if trigSlopePositive else DAQmx_Val_Falling 
            DAQmxCfgDigEdgeStartTrig(self.taskHandle,str(trig),trigEdge)
        else:
            # analog trigger
            trigSlope = DAQmx_Val_RisingSlope if trigSlopePositive else DAQmx_Val_FallingSlope 
            DAQmxCfgAnlgEdgeStartTrig(self.taskHandle,str(trig),trigSlope,trigLevel)

    def readAll(self):
        DAQmxStartTask(self.taskHandle)
        data = np.zeros((self.nCh*self.nSample,), dtype=np.float64)
#        data = AI_data_type()
        read = int32()
        timeout = 10.0
        DAQmxReadAnalogF64(self.taskHandle,self.nSample,timeout,DAQmx_Val_GroupByChannel,data,len(data),byref(read),None)
        DAQmxStopTask(self.taskHandle)
        # output data as dict
        dOut = dict()
        for n, name in enumerate(self.physicalChannel):
            dOut[name] = data[(n*self.nSample):((n+1)*self.nSample)]
        return dOut
    
    def closeAll(self):
        # close all channels
        if self.taskHandle is None:
            return
        DAQmxStopTask(self.taskHandle)
        DAQmxClearTask(self.taskHandle)  

def getTriggerSource(trigSource, trigLevel,lChName,lChDig): 
    print(trigSource)
    if trigSource == 'Immediate':
        trigSource = None
        trigSlopePositive = True
        trigLevel = 0.0
    else:
        # trig from channel
        iTrig = trigSource
        if iTrig < len(lChName):
            trigSource = lChName[iTrig]
        else:
            trigSource = lChDig[iTrig-len(lChName)]

        trigSlopePositive = True
    return trigSource, trigSlopePositive ,trigLevel

In [15]:
limit = []
lCh = []
nCh = 8
lTrace = [np.array([])]*nCh
# dt = 1.0
mAI = None
address = 'Dev1'

lChName = [("%s/ai%d" % (address, n))
                    for n in range(nCh)]

lChDig = [("%s/port0/line%d" % (address, n))
                for n in range(nCh)]

# lChDig = [("/%s/pfi0/p1.0" % (address))]

lSignalName = [('Ch%d: Data' % (n+1)) for n in range(nCh)]

chEnable = [False] * 8
chEnable[0] = True
chEnable[6] = True
for n in range(nCh):
    s = 'Ch%d: ' % (n+1)
    # check if enabled
    if chEnable[n]:  # Enable is turn or false
        limit.append([-10, 10])  # some value [-10 , 10]
        lCh.append(lChName[n])

# get config
nSample = int(200)
rate = float(10e3)
dt = 1.0/rate


trigSource = 0 # trigSource = 'Immediate' # 'Immediate' or a number between 0-7
trigLevel = 0.5
trigSource, trigSlopePositive ,trigLevel = getTriggerSource(trigSource, trigLevel,lChName,lChDig)
    
# open connection
mAI = MultiChannelAnalogInput(lCh, limit,reset = True)
mAI.configure(address, rate, nSample, trigSource, trigSlopePositive, trigLevel)

for _ in range(10):
    data = mAI.readAll()
    # put data in list of channels
    for key, data in data.items():
        indx = lChName.index(key)
        print('I am waiting during the read', indx)
        lTrace[indx] = data
        # plt.plot(np.linspace(0,dt*nSample,nSample),lTrace[indx], label='%s'%indx)
        # plt.legend()
        # plt.xlabel('Time [s]')
        # plt.ylabel('Voltage (V)')

# To close connection
mAI.closeAll()

0
200
I am waiting during the read 0
I am waiting during the read 6
I am waiting during the read 0
I am waiting during the read 6
I am waiting during the read 0
I am waiting during the read 6
I am waiting during the read 0
I am waiting during the read 6
I am waiting during the read 0
I am waiting during the read 6
I am waiting during the read 0
I am waiting during the read 6
I am waiting during the read 0
I am waiting during the read 6
I am waiting during the read 0
I am waiting during the read 6
I am waiting during the read 0
I am waiting during the read 6
I am waiting during the read 0
I am waiting during the read 6


# Some back up stuff

In [ ]:
class Driver(InstrumentDriver.InstrumentWorker):

    def performGetValue(self, quant, options={}):
        """Perform the Get Value instrument operation"""
        # only perform instrument operation for data calls
        if quant.name in self.lSignalName:
            indx = self.lSignalName.index(quant.name)
            # check if first call, if so get new traces
            if self.isFirstCall(options):
                self.getTraces()
            # return correct data
            value = quant.getTraceDict(self.lTrace[indx], dt=self.dt) # some labber thing?


    def getTraces(mAI):
        """Resample the data"""
        return lTrace